### Which citations are most frequenly issued in combination with other citations?

#### For this analysis I have narrowed the 'Citations' dataset to only those citations that have been issued to the same person, on the same day, where more than one citation was issued.  A total of 902 observations meet the above criteria.

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pyodbc
import pandas as pd
import seaborn as sns
import datetime
plt.rcParams["figure.figsize"] = [15, 7]
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

# databaseConnection = 'DRIVER={SQL Server}; SERVER=ServerName; Database=DatbaseName; UID=UserId; PWD=password;'
connection = pyodbc.connect('DRIVER={SQL Server};SERVER=WINDOWS-A6S01I2\SQLEXPRESS;DATABASE=City_of_Scottsdale;Trusted_Connection=yes;')
cursor = connection.cursor()

# SQL query to import all citation data
citations = '''
select *
from [dbo].[spd_PDCitations$] as A
inner join (select [Cited Person] as name2
			,[Cited Person Age] as age2
			,[Citation Date] as date2
			,concat([Cited Person],[Cited Person Age]) as unique_identifier
			,count(*) as count_of_citations
		   from [dbo].[spd_PDCitations$]
		   group by [Cited Person]
			,[Cited Person Age]
			,[Citation Date]
		   having count(*) >= '2'
		   ) as B on concat(A.[Cited Person],A.[Cited Person Age]) = B.unique_identifier
  '''
multiple_citations = pd.read_sql(citations, connection)
connection.close()

multiple_citations.head()

,tcmainid,Citation #,Citation Date,Citation Time,Arizona Statute Code,Charge Description,Citation Type Code,Citation Type Description,Officer Badge #,Street Number,Street,City,State,Zip,District,Beat,Zone,Cited Person,Cited Person Age,Cited Person Race,Cited Person Sex,Cited Person Ethnicity,name2,age2,date2,unique_identifier,count_of_citations
0,629619.0,8092148.0,2018-09-02,1909.0,4-251A2,Liquor-Possess Open Container In Vehicle,CT,Criminal Traffic,826.0,7xxx,E Thomas Rd,Scottsdale,AZ,NaN,D1,2.0,709.0,"Hartman, Timothy",55.0,W,M,N,"Hartman, Timothy",55.0,2018-09-02,"Hartman, Timothy55",2
1,629661.0,2065108.0,2018-09-03,208.0,28-1381A1,DUI-Impaired to Slightest Degree,CT,Criminal Traffic,515.0,7xxx,E Indian School Rd,Scottsdale,AZ,NaN,D2,6.0,809.0,"Lofthouse, Garret",21.0,W,M,N,"Lofthouse, Garret",21.0,2018-09-03,"Lofthouse, Garret21",2
2,591099.0,2018203.0,2017-10-07,2237.0,28-1381A1,DUI-Impaired to Slightest Degree,CR,Criminal,1336.0,xxx,E Happy Valley Rd / N Scottsdale Rd,Scottsdale,AZ,NaN,D4,19.0,4001.0,"Abbott, JOy",57.0,W,F,N,"Abbott, JOy",57.0,2017-10-07,"Abbott, JOy57",2
3,591100.0,2018204.0,2017-10-07,2237.0,13-2906A1,Reckless Interference-Obstruct Highway-Public Thoroughfare,CR,Criminal,1336.0,xxx,E Happy Valley Rd / N Scottsdale Rd,Scottsdale,AZ,NaN,D4,19.0,4001.0,"Abbott, JOy",57.0,W,F,N,"Abbott, JOy",57.0,2017-10-07,"Abbott, JOy57",2
4,591104.0,2045406.0,2017-10-09,113.0,28-1381A1,DUI-Impaired to Slightest Degree,CT,Criminal Traffic,515.0,1xxx,N Miller Rd,Scottsdale,AZ,NaN,D1,3.0,406.0,"Armenta-Garcia, Ever",24.0,W,M,H,"Armenta-Garcia, Ever",24.0,2017-10-09,"Armenta-Garcia, Ever24",2


In [3]:
multiple_citations.shape

(902, 27)

In [101]:
# concatenates all the citations for each person into a string within one column on one row for each person
multiple_citations_concat = multiple_citations.groupby(['Cited Person'])['Charge Description'].apply(','.join).reset_index()

multiple_citations_concat.groupby(['Charge Description']).count().sort_values('Cited Person', ascending=False).head(10)

# this isn't correct since "DUI-Impaired to Slightest Degree,No Proof of Insurance" &
# "No Proof of Insurance,DUI-Impaired to Slightest Degree" are the same thing and should be grouped together
# a sort needs to take place within the citations for each person to ensure the above doesn't take place
# see below for solution

,Cited Person
Charge Description,
"DUI-Impaired to Slightest Degree,No Proof of Insurance",13
"Shoplifting-Removal of Goods,Shoplifting-Removal of Goods",12
"DUI-Impaired to Slightest Degree,Fail to Drive In A Single Lane",10
"DUI-Impaired to Slightest Degree,Improper Right Turn Method",9
"DUI-Impaired to Slightest Degree,Failure to Control Speed to Avoid A Collision",8
"DUI-Impaired to Slightest Degree,Speed Greater Than R&P or Posted",8
"No Proof of Insurance,DUI-Impaired to Slightest Degree",8
"DUI-Impaired to Slightest Degree,Driving In Bicycle Lane",7
"Fail to Drive In A Single Lane,DUI-Impaired to Slightest Degree",6


In [97]:
# creates list for each Cited Person
multiple_citations_concat['Split'] = multiple_citations_concat['Charge Description'].apply(lambda x: x.split(',')) 

# sorts citations within list of each person
multiple_citations_concat['Split'] = multiple_citations_concat['Split'].apply(lambda x: sorted(x))

# joins list of citations back together to be able to sort in final step
multiple_citations_concat['Split'] = multiple_citations_concat['Split'].apply(','.join)

multiple_citations_concat.head()

,Cited Person,Charge Description,Split
0,"Abbott, JOy","DUI-Impaired to Slightest Degree,Reckless Interference-Obstruct Highway-Public Thoroughfare,Speed Greater Than R&P or Posted","DUI-Impaired to Slightest Degree,Reckless Interference-Obstruct Highway-Public Thoroughfare,Speed Greater Than R&P or Posted"
1,"Acevedo, Jamie","F/Obey Police Officer,DUI-Impaired to Slightest Degree","DUI-Impaired to Slightest Degree,F/Obey Police Officer"
2,"Aguilar, Francina","Release of Vehicle Violation,Suspended/Revoked D.L.","Release of Vehicle Violation,Suspended/Revoked D.L."
3,"Aiello, Jacqueline","Prescription Drug-Possess/Use,DUI-Impaired to Slightest Degree","DUI-Impaired to Slightest Degree,Prescription Drug-Possess/Use"
4,"Akmal, Jasmine","Disorderly Conduct-Disruptive Behavior,Urinate/Defecate In Public","Disorderly Conduct-Disruptive Behavior,Urinate/Defecate In Public"


In [99]:
multiple_citations_concat.groupby(['Split'])['Cited Person'] \
    .count() \
    .reset_index() \
    .sort_values('Cited Person', ascending=False) \
    .head(10)


,Split,Cited Person
105,"DUI-Impaired to Slightest Degree,No Proof of Insurance",21
84,"DUI-Impaired to Slightest Degree,Fail to Drive In A Single Lane",16
114,"DUI-Impaired to Slightest Degree,Speed Greater Than R&P or Posted",14
205,"Shoplifting-Removal of Goods,Shoplifting-Removal of Goods",12
99,"DUI-Impaired to Slightest Degree,Improper Right Turn Method",12
66,"DUI-Impaired to Slightest Degree,Driving In Bicycle Lane",11
92,"DUI-Impaired to Slightest Degree,Failure to Control Speed to Avoid A Collision",11
94,"DUI-Impaired to Slightest Degree,False Report to Law Enforcement",11
98,"DUI-Impaired to Slightest Degree,Improper Left Turn Method",9
89,"DUI-Impaired to Slightest Degree,Fail to Stop For Red Signal",9
